In [1]:
from parallel_pandas import ParallelPandas
import numpy as np
import pandas as pd
from topic2doc import get_topics

ParallelPandas.initialize(n_cpu=12)

In [2]:
data_path = "../data_input/data_corpus.csv"
df = pd.read_csv(data_path, usecols=["id", "pub_date", "region", "lang", "text_preproc1"])

In [3]:
dataemb_path = "../data_input/dataemb_corpus.csv"
df_emb = pd.read_csv(dataemb_path, usecols=["id", "emb"])

In [4]:
df_emb['lang'] = df['lang']

In [5]:
df_emb.lang.value_counts()

en    127319
de      2209
Name: lang, dtype: int64

In [6]:
def ast_eval(x):
    import numpy as np
    import torch
    import ast
    return torch.tensor(ast.literal_eval(x), dtype=torch.float32)

df_emb["emb"] = df_emb["emb"].p_apply(ast_eval)

AST_EVAL DONE:   0%|          | 0/129528 [00:00<?, ?it/s]

In [7]:
df["topics_technology"] = df_emb.p_apply(get_topics, technical=True, axis=1)

GET_TOPICS DONE:   0%|          | 0/129528 [00:00<?, ?it/s]

In [8]:
df["topics_usecase"] = df_emb[["emb", "lang"]].p_apply(get_topics, technical=False, axis=1)

GET_TOPICS DONE:   0%|          | 0/129528 [00:00<?, ?it/s]

In [9]:
df = df[(df.topics_usecase.apply(lambda x: x != [])) | (df.topics_usecase.apply(lambda x: x != []))]

In [ ]:
df.to_csv("../data_output/bertopic_result.csv", index=False)